In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
import pickle

In [2]:
data = pd.read_excel('../dataset.xls')
data.drop('Unnamed: 0', axis = 1, inplace = True)
close_value_mean =data['Close_Value'].mean()
data['Close_Value'].fillna(close_value_mean, inplace = True)
index_names = data[data['Stage'] == 'In Progress'].index
data.drop(index_names , inplace=True)
data

,Customer,Agent,SalesAgentEmailID,ContactEmailID,Stage,Product,Close_Value,Created Date,Close Date
0,Konex,Corliss Cosme,corliss@piedpiper.com,delila@konex.com,Won,GTX Plus Basic,1255.000000,2016-01-04,2016-05-24
1,Finjob,Rosalina Dieter,rosalina@piedpiper.com,belinda@finjob.com,Won,MG Special,45.000000,2016-01-04,2016-11-02
2,Kinnamplus,Donn Cantrell,donn@piedpiper.com,monte@kinnamplus.com,Lost,MG Special,1852.915504,2016-01-06,2016-12-07
4,Stanredtax,Lajuana Vencill,lajuana@piedpiper.com,candice@stanredtax.com,Won,MG Advanced,3180.000000,2016-01-07,2016-01-31
5,Hatfan,Zane Levy,zane@piedpiper.com,tyron@hatfan.com,Won,GTX Basic,517.000000,2016-01-08,2016-12-08
...,...,...,...,...,...,...,...,...,...
7293,Treequote,Darcel Schlecht,darcel@piedpiper.com,benny@treequote.com,Won,GTXPro,5085.000000,2017-12-31,2018-01-16
7294,Ganjaflex,Reed Clapper,reed@piedpiper.com,somer@ganjaflex.com,Won,GTX Basic,555.000000,2017-12-31,2018-08-29
7297,Inity,Darcel Schlecht,darcel@piedpiper.com,palmira@inity.com,Lost,GTX Plus Pro,5169.000000,2017-12-31,2018-07-10
7298,Inity,Darcel Schlecht,darcel@piedpiper.com,palmira@inity.com,Won,GTX Plus Pro,4880.000000,2017-12-31,2018-02-06


In [3]:
data['Stage'] = data['Stage'].astype('category')
data['Customer'] = data['Customer'].astype('category')
data['Agent'] = data['Agent'].astype('category')
data['SalesAgentEmailID'] = data['SalesAgentEmailID'].astype('category')
data['ContactEmailID'] = data['ContactEmailID'].astype('category')
data['Product'] = data['Product'].astype('category')

In [4]:
data['Stage']=data['Stage'].cat.codes
data['Customer']=data['Customer'].cat.codes
data['Agent']=data['Agent'].cat.codes
data['SalesAgentEmailID']=data['SalesAgentEmailID'].cat.codes
data['ContactEmailID']=data['ContactEmailID'].cat.codes
data['Product']=data['Product'].cat.codes

In [5]:
data['Created Date'] = data['Created Date'].astype(int)
data['Close Date'] = data['Close Date'].astype(int)

In [6]:
model = RandomForestClassifier()

In [7]:
X = data.drop('Stage' , axis=1)
y = data['Stage']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1) # 85% training and 15% test

In [8]:
classifired_data = model.fit(X_train,y_train)

In [9]:
f = open('RandomForest.pickle', 'wb+')
pickle.dump(classifired_data, f)
f.close()

In [10]:
y_pred = classifired_data.predict(X_test)
y_pred

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,

In [11]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("F1:",metrics.f1_score(y_test, y_pred,average='macro'))

Accuracy: 0.8471411901983664
F1: 0.8148280099293237


In [12]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.64      0.74       289
           1       0.84      0.95      0.89       568

    accuracy                           0.85       857
   macro avg       0.86      0.80      0.81       857
weighted avg       0.85      0.85      0.84       857

[[184 105]
 [ 26 542]]
